In [16]:
import pandas as pd
from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
import re
import os
import shutil
import sddk
import pickle

In [8]:
s = sddk.cloudSession(provider="sciencedata.dk", shared_folder_name="TOME/DATA/NOSCEMUS", owner="kase@zcu.cz")

connection with shared folder established with you as its ordinary user
endpoint variable has been configured to: https://sciencedata.dk/sharingout/kase%40zcu.cz/TOME/DATA/NOSCEMUS/


In [2]:
metadata_table_long = pd.read_json("../data/metadata_table_long.json")
metadata_table_long.head(5)

,Author,Full title,In,Year,Place,Publisher/Printer,Era,Form/Genre,Discipline/Content,Original,...,ids,id,date_min,date_max,filename,file_year,sents_n,words_n,disciplines_list,science_class
0,"Achrelius, Daniel",Scientiarum magnes recitatus publice anno 1690...,None,1690,[Turku],Wall,17th century,Oration,"Mathematics, Astronomy/Astrology/Cosmography, ...",Scientiarum magnes(Google Books),...,[705665],705665,1690.0,1690.0,"Achrelius,_Daniel_-_Scientiarum_magnes__Turku_...",1690.0,518,6685,"[Mathematics, Astronomy/Astrology/Cosmography,...",mixture
1,"Acidalius, Valens","Ad Iordanum Brunum Nolanum, Italum","Poematum Iani Lernutii, Iani Gulielmi, Valenti...",1603,"Liegnitz, Wrocław","Albert, David",17th century,Panegyric poem,Astronomy/Astrology/Cosmography,Ad Iordanum Brunum (1603)(CAMENA)Ad Iordanum B...,...,[801745],801745,1603.0,1603.0,Janus_Lernutius_et_al__-_Poemata__Liegnitz_160...,1603.0,6137,66744,[Astronomy/Astrology/Cosmography],exact_sciences
2,"Acosta, José de",De natura novi orbis libri duo et De promulgat...,None,1589,Salamanca,Guillelum Foquel,16th century,Monograph,"Astronomy/Astrology/Cosmography, Geography/Car...",De natura novi orbis(Biodiversity Heritage Lib...,...,[713323],713323,1589.0,1589.0,"Acosta,_José_de_-_De_natura_novi_orbis__Salama...",1589.0,9040,135979,"[Astronomy/Astrology/Cosmography, Geography/Ca...",exact_sciences
3,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",None,1620,Heidelberg,"Rosa, Geyder",17th century,Biography,Medicine,Vitae Germanorum medicorum(MDZ)Alternative lin...,...,[693148],693148,1620.0,1620.0,"Adam,_Melchior_-_Vitae_Germanorum_medicorum__H...",1620.0,16258,153868,[Medicine],life_sciences
4,"Addison, Joseph",Ad insignissimum virum dominum Thomam Burnettu...,"Examen poeticum duplex, sive, Musarum anglican...",1698,London,Richard Wellington I.,17th century,Panegyric poem,Meteorology/Earth sciences,Ad Burnettum sacrae theoriae telluris auctorem...,...,[769230],769230,1698.0,1698.0,Examen_poeticum_duplex__London_1698_pdf.txt,1698.0,3297,38011,[Meteorology/Earth sciences],exact_sciences


In [14]:
filtered_vocab_df = pd.read_json("../data/filtered_vocab_df.json")
filtered_vocab_df.head(20)

,word,1501-1550,1551-1600,1601-1650,1651-1700,mean,in_lila_embeddings,in_lasla,in_operamaiora,transl
5018,dico,42831,120825,41832,72221,69427.25,True,True,True,"say, call, tell"
6155,omnis,33152,108530,46009,66387,63519.50,True,True,True,
2515,facio,33323,109486,40134,53953,59224.00,True,True,True,"do, make, handle"
4040,pars,32591,90667,43593,51249,54525.00,True,True,True,part
6769,habeo,30884,87225,38386,57791,53571.50,True,True,True,"have, hold, possess, consider, think"
364,possum,23869,79572,36898,72892,53307.75,True,True,True,"be able, can"
897,res,27869,68790,27473,49014,43286.50,True,True,True,suddenly
4810,suus,23716,58975,33267,52441,42099.75,True,True,True,"his own, her own, its own, their own"
2392,uideo,21389,60143,28355,54288,41043.75,True,True,True,
1790,magnus,23446,61213,29708,39211,38394.50,True,True,True,"big, great"


In [4]:
vocab_freqs = filtered_vocab_df.set_index("word")["1601-1650"].to_dict()

# FastText - development and testing

In [5]:
# a genrator for iteration without flooding the CPU
class SentsCorpus:
    def __iter__(self):
        for id in ids:
            with open("../data/large_data/sents_lemmata/{}.txt".format(str(id)), "r") as f:
                sents_lemmata = [sent.strip().split() for sent in f.readlines()]
            with open("../data/large_data/sents_pos/{}.txt".format(str(id)), "r") as f:
                sents_pos = [sent.strip().split() for sent in f.readlines()]
            sents_lemmata_filtered = [] # initiate a new empty list for sentence lemmata filtered by POS tags 
            for n in range(len(sents_lemmata)):
                try:
                    pos_indices = [t[0] for t in enumerate(sents_pos[n]) if t[1] in ["PROPN", "NOUN","VERB", "ADJ"]]
                    sent_filtered = [sents_lemmata[n][i] for i in pos_indices]
                    sent_filtered = [re.sub("\W*|\d*", "", t) for t in sent_filtered]
                    yield [t for t in sent_filtered if len(t)>1]
                except:
                    pass

In [6]:
# test with a small corpus from one decade
ids = metadata_table_long[metadata_table_long["file_year"].between(1601, 1610)]["id"]
vocab_freqs = filtered_vocab_df.set_index("word")["1601-1650"].to_dict()
corpus = SentsCorpus()

In [7]:
# how many sentences are there in the small corpus?
len([s for s in corpus])

190146

In [17]:
%%time
# let's train the model
model = FastText(vector_size=100, window=10, negative=25, ns_exponent=1, sg=1, epochs=15, workers=8, min_n=5)
model.build_vocab_from_freq(word_freq=vocab_freqs)
model.train(corpus, total_examples=len([s for s in corpus]), epochs=model.epochs)

CPU times: user 3min 37s, sys: 1.03 s, total: 3min 38s
Wall time: 55.7 s


(9933198, 17796855)

In [28]:
# most similar words to harmonia
model.wv.most_similar("harmonia")

[('concentus', 0.587496817111969),
 ('motus', 0.574495255947113),
 ('parsimonia', 0.5609235167503357),
 ('notio', 0.541999876499176),
 ('incompositus', 0.5389410853385925),
 ('dissonus', 0.5348349809646606),
 ('vonia', 0.530706524848938),
 ('derideo', 0.5169634819030762),
 ('compositio', 0.5159138441085815),
 ('compos', 0.510573148727417)]

# FastText - training the main models

In [3]:
os.mkdir("../data/large_data/vectors/")

In [8]:
periods = [(1501, 1550),
                      (1551, 1600),
                      (1601, 1650),
                      (1651, 1700)]
periods_str = filtered_vocab_df.columns[1:5]
periods_str

Index(['1501-1550', '1551-1600', '1601-1650', '1651-1700'], dtype='object')

In [9]:
%%time
# main training of the models
for per_tup, per_str in zip(periods, periods_str):
    ids = metadata_table_long[metadata_table_long["file_year"].between(per_tup[0], per_tup[1])]["id"]
    vocab_freqs = filtered_vocab_df.set_index("word")[per_str].to_dict()
    corpus = SentsCorpus()
    model = FastText(vector_size=100, window=10, negative=25, ns_exponent=1, sg=1, epochs=15, workers=8, min_n=5)
    model.build_vocab_from_freq(word_freq=vocab_freqs)
    model.train(corpus, total_examples=len([s for s in corpus]), epochs=model.epochs)
    print("model for the period {} successfully trained.".format(per_str))
    model.wv.save("../data/large_data/vectors/vectors_{}.wv".format(per_str))

model for the period 1501-1550 successfully trained.
model for the period 1551-1600 successfully trained.
model for the period 1601-1650 successfully trained.
model for the period 1651-1700 successfully trained.
CPU times: user 2h 23min 2s, sys: 36.5 s, total: 2h 23min 39s
Wall time: 34min 39s


In [6]:
shutil.make_archive("../data/large_data/vectors", 'zip', "../data/large_data/vectors") #

'/Users/vojtechkase/Projects/noscemus_ETF/data/large_data/vectors.zip'

In [10]:
s.s.put("https://sciencedata.dk/sharingout/kase%40zcu.cz/TOME/DATA/NOSCEMUS/vectors.zip", data=open("../data/large_data/vectors.zip", "rb"))

<Response [201]>

In [10]:
shutil.make_archive("../data/large_data/lila_vectors", 'zip', "../data/large_data/lila_vectors") #

'/Users/vojtechkase/Projects/noscemus_ETF/data/large_data/lila_vectors.zip'

In [11]:
s.s.put("https://sciencedata.dk/sharingout/kase%40zcu.cz/TOME/DATA/NOSCEMUS/lila_vectors.zip", data=open("../data/large_data/lila_vectors.zip", "rb"))

<Response [204]>

In [15]:
periods_str = filtered_vocab_df.columns[1:5]
vectors_dict = {
    "lasla" : KeyedVectors.load_word2vec_format("../data/large_data/lila_vectors/allLASLA-lemmi-fast-100-SKIP-win5-min5.vec"),
    "operamaiora" : KeyedVectors.load_word2vec_format("../data/large_data/lila_vectors/opera-maiora-lemmas_skip_100.vec")
}
for per_str in periods_str:
    vectors_dict[per_str] = KeyedVectors.load("../data/large_data/vectors/vectors_{}.wv".format(per_str))

In [ ]:
with open("../data/large_data/vectors_dict.pkl", "wb") as file:
    pickle.dump(vectors_dict, file)

In [17]:
s.s.put("https://sciencedata.dk/sharingout/kase%40zcu.cz/TOME/DATA/NOSCEMUS/vectors_dict.pkl", data=open("../data/large_data/vectors_dict.pkl", "rb"))

<Response [201]>